<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/PFG_RNN_Predictiva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 1.7 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt
import unicodedata

In [ ]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/RNN'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

['RN_LOG.xlsx', 'RN_ACT.xlsx', 'RN_EM.xlsx', 'RN_SO.xlsx', 'RN_AT.xlsx', 'RN_SAAF.xlsx', 'df_8000_rows.xlsx']


In [ ]:
dic_dataframes = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

# Imprimir todas las claves
for clave in dic_dataframes.keys():
    print(clave)

RN_LOG.xlsx
RN_ACT.xlsx
RN_EM.xlsx
RN_SO.xlsx
RN_AT.xlsx
RN_SAAF.xlsx


In [46]:
RN_SAAF = dic_dataframes.get("RN_SAAF.xlsx")
RN_AT = dic_dataframes.get("RN_AT.xlsx")
# RN_ACT = dic_dataframes.get("RN_ACT.xlsx")
RN_EM = dic_dataframes.get("RN_EM.xlsx")
RN_SO = dic_dataframes.get("RN_SO.xlsx")
RN_LOG = dic_dataframes.get("RN_LOG.xlsx")
df_8000rows = dic_dataframes.get("df_8000_rows.xlsx")

In [148]:
# CÓDIGO SOLO PARA EL CURRO (PQ NO PUEDO ACCEDER AL DRIVE)
RN_SAAF = pd.read_excel('/content/RN_SAAF.xlsx')
RN_AT = pd.read_excel('/content/RN_AT.xlsx')
# RN_ACT = pd.read_excel('/content/RN_ACT.xlsx')
RN_EM = pd.read_excel('/content/RN_EM.xlsx')
RN_LOG = pd.read_excel('/content/RN_LOG.xlsx')
RN_SO = pd.read_excel('/content/RN_SO.xlsx')
df_8000rows = pd.read_excel('/content/df_8000_rows.xlsx')
provincia_municipio = pd.read_excel('/content/PROVINCIA_MUNICIPIO.xlsx')

In [149]:
# Convert all string columns to uppercase and remove accents/diacritics in combined_df
df_8000rows = df_8000rows.applymap(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)
provincia_municipio = provincia_municipio.applymap(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

In [150]:
# Function to add PROVINCIA column based on LOCALIDAD
def add_provincia_column(df, provincia_mapping):
    df['PROVINCIA'] = df['LOCALIDAD'].map(provincia_mapping)
    return df

# Create a dictionary mapping from LOCALIDAD to PROVINCIA
localidad_to_provincia = provincia_municipio.set_index('MUNICIPIO')['PROVINCIA'].to_dict()

# List of RN dataframes to process
rn_dataframes = {'AT': RN_AT, 'EM': RN_EM, 'LOG': RN_LOG, 'SAAF': RN_SAAF, 'SO': RN_SO}

# Add the PROVINCIA column to each RN dataframe
rn_dfs_with_provincia = {name: add_provincia_column(df, localidad_to_provincia) for name, df in rn_dataframes.items()}

# Drop rows with null values in the 'PROVINCIA' column in each RN dataframe
rn_dfs_cleaned = {name: df.dropna(subset=['PROVINCIA']) for name, df in rn_dfs_with_provincia.items()}

In [151]:
# List of columns to keep
required_columns = ['LOCALIDAD', 'GENERO', 'YEAR NACIMIENTO', 'PROVINCIA']

# Function to ensure the dataframe has all required columns
def ensure_columns(df, columns):
    for column in columns:
        if column not in df.columns:
            df[column] = np.nan  # Create column with NaN values if it does not exist
    return df[columns]  # Return dataframe with only the required columns

# Ensure all dataframes have the required columns
rn_dfs_final = {name: ensure_columns(df, required_columns) for name, df in rn_dfs_cleaned.items()}

<ipython-input-151-605fc2068a84>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = np.nan  # Create column with NaN values if it does not exist


In [152]:
# Mapping dictionary for normalizing gender values
gender_mapping = {
    'FEMENINO': 'MUJER',
    'M': 'MUJER',
    'MASCULINO': 'HOMBRE',
    'H': 'HOMBRE'
}

# Function to normalize the 'GENERO' column in a dataframe
def normalize_gender_column(df):
    df['GENERO'] = df['GENERO'].map(lambda x: gender_mapping.get(x, x))
    return df

# Apply the normalization to the 'SO' dataframe
rn_dfs_final['SO'] = normalize_gender_column(rn_dfs_final['SO'])

<ipython-input-152-333b05a14062>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['GENERO'] = df['GENERO'].map(lambda x: gender_mapping.get(x, x))


In [153]:
# Function to insert random registers into a dataframe where the 'GENERO' column is empty
def insert_random_registers(df, gender_distribution):
    # Find the indices where 'GENERO' is empty
    empty_indices = df[df['GENERO'].isna()].index

    # Generate random genders based on the given distribution for the empty indices
    genders = np.random.choice(
        ['MUJER', 'HOMBRE'],
        size=len(empty_indices),
        p=[gender_distribution['MUJER'], gender_distribution['HOMBRE']]
    )

    # Assign these random genders to the empty 'GENERO' rows
    df.loc[empty_indices, 'GENERO'] = genders
    return df

# Example of usage
gender_distribution = {'MUJER': 0.44, 'HOMBRE': 0.56}

# Insert random registers into a specific dataframe, e.g., 'ACT', where 'GENERO' is empty
rn_dfs_final['SAAF'] = insert_random_registers(rn_dfs_final['SAAF'], gender_distribution)
rn_dfs_final['SO'] = insert_random_registers(rn_dfs_final['SO'], gender_distribution)

In [154]:
# Get the min and max year in the 'YEAR NACIMIENTO' column
min_year = int(rn_dfs_final['SO']['YEAR NACIMIENTO'].min())
max_year = int(rn_dfs_final['SO']['YEAR NACIMIENTO'].max())

# Function to generate random years with the specified distribution
def generate_random_years(size):
    # 75% of the years under 2004
    under_2004 = np.random.randint(min_year, 2004, size=int(size * 0.75))
    # 25% of the years between 2004 and max_year
    over_2004 = np.random.randint(2004, max_year + 1, size=size - int(size * 0.75))
    return np.concatenate([under_2004, over_2004])

# Function to insert random years into the 'YEAR NACIMIENTO' column where it is empty
def insert_random_years(df):
    # Find the indices where 'YEAR NACIMIENTO' is empty
    empty_indices = df[df['YEAR NACIMIENTO'].isna()].index

    # Generate random years based on the specified distribution
    random_years = generate_random_years(len(empty_indices))
    np.random.shuffle(random_years)  # Shuffle to mix the values

    # Assign these random years to the empty 'YEAR NACIMIENTO' rows
    df.loc[empty_indices, 'YEAR NACIMIENTO'] = random_years
    return df

# Apply the function to the 'SO' dataframe
rn_dfs_final['SO'] = insert_random_years(rn_dfs_final['SO'])

In [155]:
# Add a column named 'SERVICIO' with the name of the dataframe to each dataframe in rn_dfs_final
def add_servicio_column(dfs):
    for name, df in dfs.items():
        df['SERVICIO'] = name
    return dfs

# Apply the function to rn_dfs_final
rn_dfs_final = add_servicio_column(rn_dfs_final)

<ipython-input-155-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICIO'] = name
<ipython-input-155-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SERVICIO'] = name
<ipython-input-155-c5260d67a514>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [156]:
# Combine all dataframes into a single dataframe
combined_df = pd.concat(rn_dfs_final.values(), ignore_index=True)

# Shuffle the rows of the combined dataframe
shuffled_combined_df = combined_df.sample(frac=1).reset_index(drop=True)

shuffled_combined_df['SERVICIO'].value_counts()

SERVICIO
SO      1457
SAAF     567
EM       203
LOG      182
AT       143
Name: count, dtype: int64

In [157]:
# Crear una función para categorizar por grupos de edad
def categorize_age(year):
    if year < 1940:
        return 'ANTES DE 1940'
    elif year < 1960:
        return '1940-1959'
    elif year < 1980:
        return '1960-1979'
    elif year < 2000:
        return '1980-1999'
    elif year < 2020:
        return '2000-2020'
    else:
        return '2020 EN ADELANTE'

rna1 = shuffled_combined_df.copy()
# Aplicar la función a la columna 'YEAR NACIMIENTO'
rna1['YEAR NACIMIENTO'] = rna1['YEAR NACIMIENTO'].apply(categorize_age)

# Ver el resultado
print(rna1)

                 LOCALIDAD  GENERO YEAR NACIMIENTO    PROVINCIA SERVICIO
0              GUADALAJARA  HOMBRE       2000-2020  GUADALAJARA     SAAF
1                 ALBACETE   MUJER       1960-1979     ALBACETE       EM
2              GUADALAJARA   MUJER       1940-1959  GUADALAJARA       SO
3              GUADALAJARA   MUJER       1980-1999  GUADALAJARA     SAAF
4              GUADALAJARA  HOMBRE       2000-2020  GUADALAJARA       SO
...                    ...     ...             ...          ...      ...
2547            MARCHAMALO  HOMBRE       2000-2020  GUADALAJARA       SO
2548            FUENSALIDA  HOMBRE       2000-2020       TOLEDO     SAAF
2549         OLIAS DEL REY  HOMBRE       2000-2020       TOLEDO      LOG
2550  CABANILLAS DEL CAMPO   MUJER       2000-2020  GUADALAJARA       SO
2551              ALBACETE   MUJER       2000-2020     ALBACETE       SO

[2552 rows x 5 columns]


##RNA predictiva con los datos reales
Vamos a realizar una primera red neuronal con los registros reales.

####Preparación de los datos

Convertimos las variables categóricas en variables dummy usando pd.get_dummies.

Separamos las características (x) de la etiqueta (y).
Convertimos la etiqueta en variables dummy para la clasificación multiclase.

In [158]:
# Convertir las variables categóricas en variables dummy
df = pd.get_dummies(rna1, columns=['LOCALIDAD', 'GENERO', 'PROVINCIA', 'YEAR NACIMIENTO'])

# Separar las características (X) y la etiqueta (y)
X = df.drop('SERVICIO', axis=1)
y = df['SERVICIO']

# Convertir la etiqueta en variables dummy
y = pd.get_dummies(y)

####División de los datos
Usaremos un 70% de los datos para entrenar y del 30% restante, un 70% para validar (21% del total) e ir ajustandio los hiperparámetros (evitando el sobreentrenamiento) y un 30% para el conjunto de test (9% del total), para evaluar el modelo y su precisión al final.

In [159]:
from sklearn.model_selection import train_test_split

# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

# Verificar las formas de los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (1786, 228)
Tamaño del conjunto de validación: (536, 228)
Tamaño del conjunto de prueba: (230, 228)


In [160]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Definir el modelo
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50
56/56 [==============================] - 2s 9ms/step - loss: 1.3029 - accuracy: 0.5274 - val_loss: 1.1578 - val_accuracy: 0.5466
Epoch 2/50
56/56 [==============================] - 0s 4ms/step - loss: 1.1046 - accuracy: 0.5773 - val_loss: 1.0903 - val_accuracy: 0.5466
Epoch 3/50
56/56 [==============================] - 0s 4ms/step - loss: 1.0593 - accuracy: 0.5761 - val_loss: 1.0584 - val_accuracy: 0.5466
Epoch 4/50
56/56 [==============================] - 0s 5ms/step - loss: 1.0324 - accuracy: 0.5840 - val_loss: 1.0364 - val_accuracy: 0.5821
Epoch 5/50
56/56 [==============================] - 0s 6ms/step - loss: 0.9947 - accuracy: 0.5952 - val_loss: 1.0035 - val_accuracy: 0.6082
Epoch 6/50
56/56 [==============================] - 0s 5ms/step - loss: 0.9656 - accuracy: 0.6181 - val_loss: 0.9662 - val_accuracy: 0.6250
Epoch 7/50
56/56 [==============================] - 0s 5ms/step - loss: 0.9384 - accuracy: 0.6243 - val_loss: 0.9543 - val_accuracy: 0.6250
Epoch 8/50
56/56 [==

In [161]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.activations import relu, tanh

# Definir el modelo con hiperparámetros ajustables
def create_model(learning_rate=0.001, activation='relu', dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation=activation))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_model(learning_rate=0.0005, activation='tanh', dropout_rate=0.3)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100
28/28 [==============================] - 2s 14ms/step - loss: 1.3689 - accuracy: 0.4882 - val_loss: 1.1677 - val_accuracy: 0.5541
Epoch 2/100
28/28 [==============================] - 0s 6ms/step - loss: 1.1059 - accuracy: 0.6092 - val_loss: 1.0652 - val_accuracy: 0.6231
Epoch 3/100
28/28 [==============================] - 0s 7ms/step - loss: 1.0172 - accuracy: 0.6333 - val_loss: 1.0182 - val_accuracy: 0.6325
Epoch 4/100
28/28 [==============================] - 0s 11ms/step - loss: 0.9718 - accuracy: 0.6445 - val_loss: 0.9784 - val_accuracy: 0.6362
Epoch 5/100
28/28 [==============================] - 0s 11ms/step - loss: 0.9328 - accuracy: 0.6585 - val_loss: 0.9619 - val_accuracy: 0.6455
Epoch 6/100
28/28 [==============================] - 0s 10ms/step - loss: 0.9122 - accuracy: 0.6601 - val_loss: 0.9529 - val_accuracy: 0.6399
Epoch 7/100
28/28 [==============================] - 0s 8ms/step - loss: 0.8889 - accuracy: 0.6657 - val_loss: 0.9451 - val_accuracy: 0.6269
Epoch 8/1

In [162]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Definir el modelo con hiperparámetros ajustables
def create_simple_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_simple_model(learning_rate=0.001, dropout_rate=0.3)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50
56/56 [==============================] - 2s 11ms/step - loss: 1.3841 - accuracy: 0.5050 - val_loss: 1.1728 - val_accuracy: 0.5466
Epoch 2/50
56/56 [==============================] - 0s 4ms/step - loss: 1.1172 - accuracy: 0.5795 - val_loss: 1.0901 - val_accuracy: 0.5466
Epoch 3/50
56/56 [==============================] - 0s 3ms/step - loss: 1.0536 - accuracy: 0.5941 - val_loss: 1.0338 - val_accuracy: 0.6157
Epoch 4/50
56/56 [==============================] - 0s 3ms/step - loss: 0.9935 - accuracy: 0.6237 - val_loss: 0.9955 - val_accuracy: 0.6474
Epoch 5/50
56/56 [==============================] - 0s 4ms/step - loss: 0.9580 - accuracy: 0.6405 - val_loss: 0.9709 - val_accuracy: 0.6549
Epoch 6/50
56/56 [==============================] - 0s 4ms/step - loss: 0.9312 - accuracy: 0.6411 - val_loss: 0.9499 - val_accuracy: 0.6418
Epoch 7/50
56/56 [==============================] - 0s 4ms/step - loss: 0.9407 - accuracy: 0.6456 - val_loss: 0.9432 - val_accuracy: 0.6437
Epoch 8/50
56/56 [=

###Modelo XGBoost

In [163]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Codificar las etiquetas usando LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.idxmax(axis=1))  # Convertir de one-hot encoding a etiquetas enteras

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Crear y entrenar el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))
xgb_model.fit(X_train_xgb, y_train_xgb)

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = xgb_model.predict_proba(X_train_xgb)
X_test_nn = xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Definir el modelo de red neuronal
def create_nn_model(input_dim, learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train_nn.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo de red neuronal
nn_model = create_nn_model(input_dim=X_train_nn.shape[1])

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo de red neuronal
history = nn_model.fit(X_train_nn, y_train_nn, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = nn_model.evaluate(X_test_nn, y_test_nn)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100
23/23 [==============================] - 1s 14ms/step - loss: 1.4170 - accuracy: 0.5420 - val_loss: 1.1813 - val_accuracy: 0.5642
Epoch 2/100
23/23 [==============================] - 0s 4ms/step - loss: 1.0717 - accuracy: 0.5917 - val_loss: 0.9531 - val_accuracy: 0.6508
Epoch 3/100
23/23 [==============================] - 0s 5ms/step - loss: 0.9300 - accuracy: 0.6821 - val_loss: 0.8578 - val_accuracy: 0.7039
Epoch 4/100
23/23 [==============================] - 0s 5ms/step - loss: 0.8603 - accuracy: 0.7017 - val_loss: 0.8187 - val_accuracy: 0.7095
Epoch 5/100
23/23 [==============================] - 0s 5ms/step - loss: 0.8244 - accuracy: 0.7024 - val_loss: 0.8009 - val_accuracy: 0.7123
Epoch 6/100
23/23 [==============================] - 0s 6ms/step - loss: 0.8132 - accuracy: 0.7059 - val_loss: 0.7898 - val_accuracy: 0.7179
Epoch 7/100
23/23 [==============================] - 0s 6ms/step - loss: 0.7962 - accuracy: 0.7129 - val_loss: 0.7823 - val_accuracy: 0.7235
Epoch 8/100


In [164]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 [==============================] - 0s 72ms/step
Original Servicio: SO
Servicio: AT, Probabilidad: 0.0001
Servicio: EM, Probabilidad: 0.0058
Servicio: LOG, Probabilidad: 0.0036
Servicio: SAAF, Probabilidad: 0.0221
Servicio: SO, Probabilidad: 0.9684


In [165]:
import numpy as np

# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[22]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[22]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 [==============================] - 0s 22ms/step
Original Servicio: EM
Servicio: AT, Probabilidad: 0.3682
Servicio: EM, Probabilidad: 0.5760
Servicio: LOG, Probabilidad: 0.0043
Servicio: SAAF, Probabilidad: 0.0198
Servicio: SO, Probabilidad: 0.0317


In [166]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[70]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[70]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 [==============================] - 0s 20ms/step
Original Servicio: AT
Servicio: AT, Probabilidad: 0.1020
Servicio: EM, Probabilidad: 0.0166
Servicio: LOG, Probabilidad: 0.2469
Servicio: SAAF, Probabilidad: 0.1943
Servicio: SO, Probabilidad: 0.4403


##RNA predictiva con datos generados
Red Neuronal con 8000 registros más que han sido creados teniendo en cuenta las estadísticas reales de los datos, es decir, contemplando cómo se distribuyen por edad, género y localidad y, obviamente, el municipio y la provincia correctamente computados.

Esto se hace para intentar aumentar el porcentaje de precisión obtenido.

In [167]:
# Calculate the frequency of each element in the 'SERVICIO' column
service_frequency = df['SERVICIO'].value_counts()

# Group by 'PROVINCIA' and 'SERVICIO' and count the occurrences
service_province_freq = shuffled_combined_df.groupby(['PROVINCIA', 'SERVICIO']).size().reset_index(name='COUNT')

# Function to assign services based on frequency
def assign_service(row, freq_df):
    province = row['PROVINCIA']
    possible_services = freq_df[freq_df['PROVINCIA'] == province]
    if possible_services.empty:
        return np.nan
    else:
        return np.random.choice(possible_services['SERVICIO'].values, p=possible_services['COUNT'].values / possible_services['COUNT'].sum())

# Apply the function to assign services
df_8000rows['SERVICIO'] = df_8000rows.apply(assign_service, axis=1, args=(service_province_freq,))

# Ensure balanced classes by sampling
min_class_size = service_frequency.min()
balanced_df = df_8000rows.groupby('SERVICIO').apply(lambda x: x.sample(n=min_class_size, replace=True)).reset_index(drop=True)

In [168]:
# Calculate the frequency of each service
service_frequency = shuffled_combined_df['SERVICIO'].value_counts()

# Set the target number of records per service
target_size = 1457

# Create a balanced dataframe starting with the initial shuffled data
to_use_df = shuffled_combined_df.copy()

# Loop through each service to balance the dataframe
for service, count in service_frequency.items():
    if count < target_size:
        # Get the rows for the current service
        service_df = balanced_df[balanced_df['SERVICIO'] == service]
        # Sample additional rows with replacement to reach the target size
        additional_samples = service_df.sample(n=target_size - count, replace=True)
        to_use_df = pd.concat([to_use_df, additional_samples])

# Shuffle the final balanced dataframe
to_use_df = to_use_df.sample(frac=1).reset_index(drop=True)

In [169]:
# Shuffle the rows of the dataframe ready to use
to_use_df = to_use_df.sample(frac=1).reset_index(drop=True)

In [170]:
# Get the min and max year in the 'YEAR NACIMIENTO' column
min_year = int(to_use_df['YEAR NACIMIENTO'].min())
max_year = int(to_use_df['YEAR NACIMIENTO'].max())

# Function to generate random years with the specified distribution
def generate_random_years(size):
    return np.random.randint(min_year, max_year + 1, size=int(size))

# Function to insert random years into the 'YEAR NACIMIENTO' column where it is empty
def insert_random_years(df):
    # Find the indices where 'YEAR NACIMIENTO' is empty
    empty_indices = df[df['YEAR NACIMIENTO'].isna()].index

    # Generate random years based on the specified distribution
    random_years = generate_random_years(len(empty_indices))

    # Assign these random years to the empty 'YEAR NACIMIENTO' rows
    df.loc[empty_indices, 'YEAR NACIMIENTO'] = random_years
    return df

# Apply the function to the dataframe
to_use_df = insert_random_years(to_use_df)

In [171]:
# Crear una función para categorizar por grupos de edad
def categorize_age(year):
    if year < 1940:
        return 'Antes de 1940'
    elif year < 1960:
        return '1940-1959'
    elif year < 1980:
        return '1960-1979'
    elif year < 2000:
        return '1980-1999'
    else:
        return '2000 en adelante'

# Aplicar la función a la columna 'YEAR NACIMIENTO'
to_use_df['YEAR NACIMIENTO'] = to_use_df['YEAR NACIMIENTO'].apply(categorize_age)

# Ver el resultado
print(to_use_df.head())

                LOCALIDAD  GENERO   YEAR NACIMIENTO    PROVINCIA SERVICIO
0   VILLANUEVA DE LA JARA   MUJER         1980-1999       CUENCA      LOG
1               VALDEMECA   MUJER         1960-1979       CUENCA      LOG
2  VILLAMAYOR DE SANTIAGO   MUJER  2000 en adelante       CUENCA      LOG
3      CIRUELOS DEL PINAR   MUJER         1980-1999  GUADALAJARA      LOG
4          LA PUEBLANUEVA  HOMBRE         1960-1979       TOLEDO       AT


In [172]:
# Determina la cantidad de servicios únicos
unique_services = to_use_df['SERVICIO'].unique()
num_services = len(unique_services)

# Determina cuántos registros por servicio necesitas para el conjunto de prueba
test_size_per_service = 85 // num_services

# Si hay más de 85 registros, ajusta la muestra para que esté balanceada
if test_size_per_service * num_services < 85:
    additional_samples = 85 - (test_size_per_service * num_services)
else:
    additional_samples = 0

# Crear el conjunto de prueba balanceado
test_df = pd.DataFrame()

for service in unique_services:
    service_df = to_use_df[to_use_df['SERVICIO'] == service]
    test_samples = service_df.sample(n=test_size_per_service, replace=False)
    test_df = pd.concat([test_df, test_samples])

# Añadir muestras adicionales si es necesario
if additional_samples > 0:
    additional_df = to_use_df[~to_use_df.index.isin(test_df.index)]
    additional_samples_df = additional_df.sample(n=additional_samples, replace=False)
    test_df = pd.concat([test_df, additional_samples_df])

# Eliminar las filas seleccionadas del dataframe original para crear el conjunto de entrenamiento/validación
to_use_df = to_use_df.drop(test_df.index)

# Verifica el conjunto de prueba
print(test_df['SERVICIO'].value_counts())
print(test_df.shape)

# Verifica el conjunto de entrenamiento/validación
print(to_use_df.shape)

SERVICIO
LOG     17
AT      17
SAAF    17
EM      17
SO      17
Name: count, dtype: int64
(85, 5)
(7200, 5)


#Red Neuronal Predictiva

Creación de una red neuronal predictiva que sea capaz de clasificar nuevos usuarios con buena precisión en servicio y provincia.

In [173]:
# Convertir las variables categóricas en variables dummy
df = pd.get_dummies(to_use_df, columns=['LOCALIDAD', 'GENERO', 'PROVINCIA', 'YEAR NACIMIENTO'])

# Separar las características (X) y la etiqueta (y)
X = df.drop('SERVICIO', axis=1)
y = df['SERVICIO']

# Convertir la etiqueta en variables dummy
y = pd.get_dummies(y)

In [174]:
# Dividir los datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.3, random_state=42)

# Verificar las formas de los conjuntos de datos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (5040, 504)
Tamaño del conjunto de validación: (1512, 504)
Tamaño del conjunto de prueba: (648, 504)


In [175]:
# Definir el modelo
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compilar el modelo
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50
158/158 [==============================] - 2s 9ms/step - loss: 1.5199 - accuracy: 0.3200 - val_loss: 1.3087 - val_accuracy: 0.4597
Epoch 2/50
158/158 [==============================] - 1s 7ms/step - loss: 1.2506 - accuracy: 0.4843 - val_loss: 1.0200 - val_accuracy: 0.6620
Epoch 3/50
158/158 [==============================] - 1s 6ms/step - loss: 1.0001 - accuracy: 0.6266 - val_loss: 0.7826 - val_accuracy: 0.7110
Epoch 4/50
158/158 [==============================] - 1s 5ms/step - loss: 0.8100 - accuracy: 0.7127 - val_loss: 0.6498 - val_accuracy: 0.7679
Epoch 5/50
158/158 [==============================] - 1s 5ms/step - loss: 0.6941 - accuracy: 0.7565 - val_loss: 0.5996 - val_accuracy: 0.7831
Epoch 6/50
158/158 [==============================] - 1s 5ms/step - loss: 0.6266 - accuracy: 0.7760 - val_loss: 0.5440 - val_accuracy: 0.8056
Epoch 7/50
158/158 [==============================] - 1s 5ms/step - loss: 0.5906 - accuracy: 0.7905 - val_loss: 0.5291 - val_accuracy: 0.8122
Epoch 

In [176]:
# Definir el modelo con hiperparámetros ajustables
def create_model(learning_rate=0.001, activation='relu', dropout_rate=0.5):
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation=activation))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_model(learning_rate=0.0005, activation='tanh', dropout_rate=0.3)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_val, y_val))

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100
79/79 [==============================] - 2s 9ms/step - loss: 1.4881 - accuracy: 0.3671 - val_loss: 1.3354 - val_accuracy: 0.4821
Epoch 2/100
79/79 [==============================] - 1s 6ms/step - loss: 1.2109 - accuracy: 0.5437 - val_loss: 1.0629 - val_accuracy: 0.6243
Epoch 3/100
79/79 [==============================] - 0s 6ms/step - loss: 0.9678 - accuracy: 0.6528 - val_loss: 0.8487 - val_accuracy: 0.6984
Epoch 4/100
79/79 [==============================] - 0s 6ms/step - loss: 0.7920 - accuracy: 0.7107 - val_loss: 0.7383 - val_accuracy: 0.7209
Epoch 5/100
79/79 [==============================] - 0s 6ms/step - loss: 0.7037 - accuracy: 0.7435 - val_loss: 0.6984 - val_accuracy: 0.7295
Epoch 6/100
79/79 [==============================] - 1s 7ms/step - loss: 0.6521 - accuracy: 0.7520 - val_loss: 0.6680 - val_accuracy: 0.7368
Epoch 7/100
79/79 [==============================] - 0s 6ms/step - loss: 0.6243 - accuracy: 0.7554 - val_loss: 0.6573 - val_accuracy: 0.7427
Epoch 8/100
7

In [177]:
# Definir el modelo con hiperparámetros ajustables
def create_simple_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo con hiperparámetros ajustados
model = create_simple_model(learning_rate=0.001, dropout_rate=0.3)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/50
158/158 [==============================] - 2s 5ms/step - loss: 1.5501 - accuracy: 0.3018 - val_loss: 1.3675 - val_accuracy: 0.4630
Epoch 2/50
158/158 [==============================] - 1s 4ms/step - loss: 1.2482 - accuracy: 0.4877 - val_loss: 1.0258 - val_accuracy: 0.6085
Epoch 3/50
158/158 [==============================] - 1s 3ms/step - loss: 0.9827 - accuracy: 0.6276 - val_loss: 0.7957 - val_accuracy: 0.7235
Epoch 4/50
158/158 [==============================] - 1s 5ms/step - loss: 0.8101 - accuracy: 0.7141 - val_loss: 0.6723 - val_accuracy: 0.7705
Epoch 5/50
158/158 [==============================] - 1s 5ms/step - loss: 0.6938 - accuracy: 0.7486 - val_loss: 0.6155 - val_accuracy: 0.7784
Epoch 6/50
158/158 [==============================] - 1s 6ms/step - loss: 0.6266 - accuracy: 0.7752 - val_loss: 0.5665 - val_accuracy: 0.7956
Epoch 7/50
158/158 [==============================] - 1s 5ms/step - loss: 0.5858 - accuracy: 0.7893 - val_loss: 0.5564 - val_accuracy: 0.7989
Epoch 

Ejemplo de prueba para ver qué tan bien predice.

In [178]:
import numpy as np

# Ejemplo de un registro individual (preprocesado)
# Asegúrate de que el registro individual tenga las mismas características que los datos de entrenamiento y prueba
new_record = X_test.iloc[3]

# Obtener el servicio objetivo original
original_service = y_test.iloc[3].idxmax()

 # Convertir el nuevo registro a un DataFrame
new_record_df = pd.DataFrame([new_record])

# Asegurarse de que tenga las mismas columnas que el DataFrame de entrenamiento
missing_cols = set(X.columns) - set(new_record_df.columns)
for col in missing_cols:
    new_record_df[col] = 0
new_record_df = new_record_df[X.columns]

# Convertir a numpy array
new_record_array = new_record_df.values

# Predecir usando el modelo entrenado
predictions = model.predict(new_record_array)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Imprimir el servicio objetivo original y las probabilidades
print(f"Original Servicio: {original_service}")
# Imprimir las probabilidades
service_labels = y.columns  # Asegúrate de que 'y' tiene las columnas de las categorías de servicio
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 [==============================] - 0s 77ms/step
Original Servicio: LOG
Servicio: AT, Probabilidad: 0.3234
Servicio: EM, Probabilidad: 0.0106
Servicio: LOG, Probabilidad: 0.6517
Servicio: SAAF, Probabilidad: 0.0122
Servicio: SO, Probabilidad: 0.0021


In [179]:
import numpy as np

# Ejemplo de un registro individual (preprocesado)
# Asegúrate de que el registro individual tenga las mismas características que los datos de entrenamiento y prueba
new_record = X_test.iloc[22]

# Obtener el servicio objetivo original
original_service = y_test.iloc[22].idxmax()

 # Convertir el nuevo registro a un DataFrame
new_record_df = pd.DataFrame([new_record])

# Asegurarse de que tenga las mismas columnas que el DataFrame de entrenamiento
missing_cols = set(X.columns) - set(new_record_df.columns)
for col in missing_cols:
    new_record_df[col] = 0
new_record_df = new_record_df[X.columns]

# Convertir a numpy array
new_record_array = new_record_df.values

# Predecir usando el modelo entrenado
predictions = model.predict(new_record_array)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Imprimir el servicio objetivo original y las probabilidades
print(f"Original Servicio: {original_service}")
# Imprimir las probabilidades
service_labels = y.columns  # Asegúrate de que 'y' tiene las columnas de las categorías de servicio
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 [==============================] - 0s 22ms/step
Original Servicio: LOG
Servicio: AT, Probabilidad: 0.0235
Servicio: EM, Probabilidad: 0.0163
Servicio: LOG, Probabilidad: 0.9197
Servicio: SAAF, Probabilidad: 0.0141
Servicio: SO, Probabilidad: 0.0263


In [180]:
import numpy as np

# Ejemplo de un registro individual (preprocesado)
# Asegúrate de que el registro individual tenga las mismas características que los datos de entrenamiento y prueba
new_record = X_test.iloc[70]

# Obtener el servicio objetivo original
original_service = y_test.iloc[70].idxmax()

 # Convertir el nuevo registro a un DataFrame
new_record_df = pd.DataFrame([new_record])

# Asegurarse de que tenga las mismas columnas que el DataFrame de entrenamiento
missing_cols = set(X.columns) - set(new_record_df.columns)
for col in missing_cols:
    new_record_df[col] = 0
new_record_df = new_record_df[X.columns]

# Convertir a numpy array
new_record_array = new_record_df.values

# Predecir usando el modelo entrenado
predictions = model.predict(new_record_array)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Imprimir el servicio objetivo original y las probabilidades
print(f"Original Servicio: {original_service}")
# Imprimir las probabilidades
service_labels = y.columns  # Asegúrate de que 'y' tiene las columnas de las categorías de servicio
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 [==============================] - 0s 20ms/step
Original Servicio: LOG
Servicio: AT, Probabilidad: 0.0000
Servicio: EM, Probabilidad: 0.0000
Servicio: LOG, Probabilidad: 0.9999
Servicio: SAAF, Probabilidad: 0.0001
Servicio: SO, Probabilidad: 0.0000


In [189]:
pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 25.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [188]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

# Definir el modelo con hiperparámetros ajustables
def create_nn_model(learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el KerasClassifier con SciKeras
nn_model = KerasClassifier(model=create_nn_model, verbose=0)

# Definir los hiperparámetros para la búsqueda aleatoria
param_dist_nn = {
    'model__learning_rate': uniform(0.0001, 0.001),
    'model__dropout_rate': uniform(0.1, 0.5),
    'epochs': [50, 100, 200],
    'batch_size': [16, 32, 64]
}

# Realizar la búsqueda aleatoria
random_search_nn = RandomizedSearchCV(estimator=nn_model, param_distributions=param_dist_nn, scoring='accuracy', cv=3, n_iter=20, verbose=1, random_state=42)
random_search_nn.fit(X_train, y_train)

# Obtener el mejor modelo
best_nn_model = random_search_nn.best_estimator_

# Evaluar el mejor modelo de red neuronal
loss, accuracy = best_nn_model.score(X_test, y_test)
print(f'Best Neural Network Model Accuracy: {accuracy}')

ModuleNotFoundError: No module named 'tensorflow.keras.wrappers'

##Modelo XGBoost con datos generados

In [181]:
# Codificar las etiquetas usando LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y.idxmax(axis=1))  # Convertir de one-hot encoding a etiquetas enteras

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_xgb, X_test_xgb, y_train_xgb, y_test_xgb = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

# Crear y entrenar el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))
xgb_model.fit(X_train_xgb, y_train_xgb)

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = xgb_model.predict_proba(X_train_xgb)
X_test_nn = xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Definir el modelo de red neuronal
def create_nn_model(input_dim, learning_rate=0.001, dropout_rate=0.3):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(y_train_nn.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo de red neuronal
nn_model = create_nn_model(input_dim=X_train_nn.shape[1])

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo de red neuronal
history = nn_model.fit(X_train_nn, y_train_nn, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = nn_model.evaluate(X_test_nn, y_test_nn)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100
63/63 [==============================] - 2s 8ms/step - loss: 1.2614 - accuracy: 0.5637 - val_loss: 0.7179 - val_accuracy: 0.8065
Epoch 2/100
63/63 [==============================] - 0s 3ms/step - loss: 0.6532 - accuracy: 0.8043 - val_loss: 0.5719 - val_accuracy: 0.8016
Epoch 3/100
63/63 [==============================] - 0s 3ms/step - loss: 0.6045 - accuracy: 0.8038 - val_loss: 0.5516 - val_accuracy: 0.8075
Epoch 4/100
63/63 [==============================] - 0s 3ms/step - loss: 0.5849 - accuracy: 0.8051 - val_loss: 0.5444 - val_accuracy: 0.8095
Epoch 5/100
63/63 [==============================] - 0s 3ms/step - loss: 0.5748 - accuracy: 0.8033 - val_loss: 0.5357 - val_accuracy: 0.8075
Epoch 6/100
63/63 [==============================] - 0s 3ms/step - loss: 0.5668 - accuracy: 0.8098 - val_loss: 0.5356 - val_accuracy: 0.8125
Epoch 7/100
63/63 [==============================] - 0s 4ms/step - loss: 0.5640 - accuracy: 0.8083 - val_loss: 0.5324 - val_accuracy: 0.8135
Epoch 8/100
6

In [182]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 [==============================] - 0s 77ms/step
Original Servicio: EM
Servicio: AT, Probabilidad: 0.8002
Servicio: EM, Probabilidad: 0.0716
Servicio: LOG, Probabilidad: 0.0007
Servicio: SAAF, Probabilidad: 0.0745
Servicio: SO, Probabilidad: 0.0530


In [183]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 [==============================] - 0s 20ms/step
Original Servicio: EM
Servicio: AT, Probabilidad: 0.8002
Servicio: EM, Probabilidad: 0.0716
Servicio: LOG, Probabilidad: 0.0007
Servicio: SAAF, Probabilidad: 0.0745
Servicio: SO, Probabilidad: 0.0530


In [184]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")


1/1 [==============================] - 0s 22ms/step
Original Servicio: EM
Servicio: AT, Probabilidad: 0.8002
Servicio: EM, Probabilidad: 0.0716
Servicio: LOG, Probabilidad: 0.0007
Servicio: SAAF, Probabilidad: 0.0745
Servicio: SO, Probabilidad: 0.0530


In [185]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Definir el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))

# Definir los hiperparámetros para la búsqueda en cuadrícula
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Realizar la búsqueda en cuadrícula
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1)
grid_search.fit(X_train_xgb, y_train_xgb)

# Obtener el mejor modelo
best_xgb_model = grid_search.best_estimator_

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = best_xgb_model.predict_proba(X_train_xgb)
X_test_nn = best_xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Evaluar el mejor modelo XGBoost
best_xgb_accuracy = grid_search.best_score_
print(f'Best XGBoost Model Accuracy: {best_xgb_accuracy}')

Fitting 3 folds for each of 144 candidates, totalling 432 fits
Best XGBoost Model Accuracy: 0.7236111111111111


In [ ]:
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint

# Definir el modelo XGBoost
xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=len(label_encoder.classes_))

# Definir los hiperparámetros para la búsqueda aleatoria
param_dist = {
    'n_estimators': randint(50, 300),
    'max_depth': randint(3, 10),
    'learning_rate': uniform(0.01, 0.3),
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3)
}

# Realizar la búsqueda aleatoria
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, scoring='accuracy', cv=3, n_iter=50, verbose=1, random_state=42)
random_search.fit(X_train_xgb, y_train_xgb)

# Obtener el mejor modelo
best_xgb_model = random_search.best_estimator_

# Obtener las predicciones de probabilidad del conjunto de prueba
X_train_nn = best_xgb_model.predict_proba(X_train_xgb)
X_test_nn = best_xgb_model.predict_proba(X_test_xgb)

# Convertir las etiquetas del conjunto de prueba a one-hot encoding
y_train_nn = to_categorical(y_train_xgb, num_classes=len(label_encoder.classes_))
y_test_nn = to_categorical(y_test_xgb, num_classes=len(label_encoder.classes_))

# Evaluar el mejor modelo XGBoost
best_xgb_accuracy = random_search.best_score_
print(f'Best XGBoost Model Accuracy: {best_xgb_accuracy}')

In [186]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2i

# Definir el modelo de red neuronal con regularización y Dropout
def create_nn_model(input_dim, learning_rate=0.001, dropout_rate=0.3, regularization=0.01):
    model = Sequential()
    model.add(Dense(128, input_dim=input_dim, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(32, activation='relu', kernel_regularizer=l2(regularization)))
    model.add(Dense(y_train_nn.shape[1], activation='softmax'))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

# Crear el modelo de red neuronal
nn_model = create_nn_model(input_dim=X_train_nn.shape[1], learning_rate=0.001, dropout_rate=0.4, regularization=0.01)

# Añadir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo de red neuronal
history = nn_model.fit(X_train_nn, y_train_nn, epochs=100, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

# Evaluar el modelo en el conjunto de prueba
loss, accuracy = nn_model.evaluate(X_test_nn, y_test_nn)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100
63/63 [==============================] - 1s 6ms/step - loss: 2.3895 - accuracy: 0.5166 - val_loss: 1.7545 - val_accuracy: 0.8125
Epoch 2/100
63/63 [==============================] - 0s 3ms/step - loss: 1.4217 - accuracy: 0.7946 - val_loss: 1.0949 - val_accuracy: 0.8274
Epoch 3/100
63/63 [==============================] - 0s 4ms/step - loss: 1.0868 - accuracy: 0.8108 - val_loss: 0.9436 - val_accuracy: 0.8284
Epoch 4/100
63/63 [==============================] - 0s 4ms/step - loss: 0.9774 - accuracy: 0.8123 - val_loss: 0.8735 - val_accuracy: 0.8214
Epoch 5/100
63/63 [==============================] - 0s 3ms/step - loss: 0.9170 - accuracy: 0.8115 - val_loss: 0.8390 - val_accuracy: 0.8185
Epoch 6/100
63/63 [==============================] - 0s 5ms/step - loss: 0.8740 - accuracy: 0.8224 - val_loss: 0.8020 - val_accuracy: 0.8264
Epoch 7/100
63/63 [==============================] - 0s 3ms/step - loss: 0.8652 - accuracy: 0.8140 - val_loss: 0.7863 - val_accuracy: 0.8254
Epoch 8/100
6

In [187]:
# Ejemplo de un registro individual (preprocesado) con el conjunto de prueba de XGBoost
new_record_xgb = X_test_xgb.iloc[3]

# Convertir el nuevo registro a un DataFrame
new_record_xgb_df = pd.DataFrame([new_record_xgb])

# Obtener las predicciones de probabilidad del modelo XGBoost
new_record_nn_input = best_xgb_model.predict_proba(new_record_xgb_df)

# Predecir usando el modelo de red neuronal entrenado
predictions = nn_model.predict(new_record_nn_input)

# Obtener las probabilidades para cada tipo de servicio
service_probabilities = predictions[0]

# Obtener el servicio objetivo original
original_service = y_test_xgb[3]  # Indexar directamente el array de NumPy

# Convertir el servicio objetivo original a etiqueta de texto
original_service_label = label_encoder.inverse_transform([original_service])[0]

# Imprimir el servicio objetivo original y las probabilidades
service_labels = label_encoder.classes_  # Obtener las etiquetas originales
print(f"Original Servicio: {original_service_label}")
for label, prob in zip(service_labels, service_probabilities):
    print(f"Servicio: {label}, Probabilidad: {prob:.4f}")

1/1 [==============================] - 0s 75ms/step
Original Servicio: EM
Servicio: AT, Probabilidad: 0.8878
Servicio: EM, Probabilidad: 0.0211
Servicio: LOG, Probabilidad: 0.0067
Servicio: SAAF, Probabilidad: 0.0446
Servicio: SO, Probabilidad: 0.0398
